In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
from tqdm.auto import tqdm
from dataset import get_labelled, get_unlabelled, _preload
from Commit import CommitFactory
from Model import CommitDiffModelFactory

In [2]:
CONTEXT_SIZE = 16
BAG_SIZE = 512
OUTPUT_SIZE = 128

In [3]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [4]:
_preload()

Loading Commit lookup table


  0%|          | 0/4 [00:00<?, ?it/s]

Loading file ../data/commit_lookups/supervised_commit_data_lookup0-1000.pickle
Appending pickle of length: 184 , new dict length: 184
Loading file ../data/commit_lookups/commit_data_lookup12500-15000.pickle
Appending pickle of length: 2497 , new dict length: 2681
Loading file ../data/commit_lookups/commit_data_lookup10000-12500.pickle
Appending pickle of length: 2496 , new dict length: 5177


In [5]:
X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Positive X_train:   0%|          | 0/97 [00:00<?, ?it/s]

Generating Positive y_train:   0%|          | 0/97 [00:00<?, ?it/s]

Generating Negative X_train:   0%|          | 0/70 [00:00<?, ?it/s]

Generating Negative y_train:   0%|          | 0/70 [00:00<?, ?it/s]

In [6]:
X_train_unsupervised = get_unlabelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Unsupervised X_train:   0%|          | 0/5177 [00:00<?, ?it/s]

In [7]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))
print("Unsupervised Train Size", len(X_train_unsupervised))

Train set size 132
Train set split 0.5833333333333334
Test set size 33
Test set split 0.5454545454545454
Unsupervised Train Size 5177


In [8]:
debug = CommitDiffModel()
debug.initialize(encoder=0)
debug.fit_siam(np.array(X_train_unsupervised), epochs=1, batch_size=64, verbose=1)
debug.fit_binary_classification(X_train, np.array(y_train), epochs=1, batch_size=64, verbose=1)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
3/3 [==============================] - 7s 48ms/step - loss: 0.8546


In [9]:
output = debug.debug(X_train)
print("Output:", output)

Encoded1: tf.Tensor(
[[ 0.5313834   0.31119585 -0.40491328  0.22323953 -0.9756087  -1.3600199
   1.2041396  -0.7061452   0.07178355  0.3344193  -0.01434582  0.35798025
   1.2776144   0.9289185   1.163372   -0.68175316  0.79743826 -0.42700344
   0.5562434   0.48803836 -0.21568955  0.45876527 -0.47208247 -0.45189223
  -0.6977732  -0.68747723  0.31950048  0.4046296  -0.14091158 -0.4077265
  -0.91834944 -0.41066644 -0.45000833 -0.93571633 -0.48524076 -0.43913317
  -0.66083956 -0.48078325 -0.08665306  0.05938193 -0.87294376  0.48737195
  -0.73684     0.7643501  -0.3570439   0.94443005 -0.24262922 -0.2797534
   0.05649977 -0.3166979  -0.2671985   0.669685   -0.60511714  0.00990672
  -0.98669547  0.9271621  -0.27688527  0.4719644  -0.17257495  0.34804717
  -0.47625577 -0.95920277  0.28417453 -0.0234936   0.45645216 -0.6497511
  -0.19192824  0.17270252  0.19320747 -0.08592993 -0.17439823  0.03072764
  -0.02064663 -0.22217181  0.07262839  0.35823387 -0.3924521  -0.6320791
  -0.5909932   0.73460

InvalidArgumentError: Exception encountered when calling layer 'reshape' (type Reshape).

{{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 0 of dimension 0 out of bounds. [Op:StridedSlice] name: reshape/strided_slice/

Call arguments received by layer 'reshape' (type Reshape):
  • inputs=1

In [10]:
for encoder in tqdm([6,7,8,9,10,11,1,2,3,4]):
    model = CommitDiffModel()
    model.initialize(encoder)
    model.fit_siam(np.array(X_train_unsupervised), epochs=4, batch_size=256, verbose=1)
    model.fit_binary_classification(X_train, np.array(y_train), epochs=8, batch_size=32, verbose=1)
    score = model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)
    print("Enocder:", encoder)
    print("Score:", score)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/4
21/21 [==============================] - 3s 96ms/step - loss: -0.3797
Epoch 2/4
21/21 [==============================] - 2s 80ms/step - loss: -0.5845
Epoch 3/4
21/21 [==============================] - 2s 88ms/step - loss: -0.6319
Epoch 4/4
21/21 [==============================] - 2s 83ms/step - loss: -0.6629
Epoch 1/8


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/model_5/concatenate_1/ConcatOffset' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/brennan/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/brennan/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/brennan/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/brennan/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    